In [645]:
import json

### Dataset

In [574]:
dataset_name = ["nfcorpus", "trec-covid", "trec-covid-v2", "climate-fever"][3]

In [575]:
import json
with open(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/queries.jsonl") as f:
    queries = [json.loads(i) for i in f.readlines()]
    queries_dict = {}
    for q in queries:
        queries_dict[q["_id"]] = q["text"]
        
import json
with open(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/corpus.jsonl") as f:
    corpus = [json.loads(i) for i in f.readlines()]
    corpus_dict = {}
    for c in corpus:
        corpus_dict[c["_id"]] = c["text"]


In [576]:
import pandas as pd
qrels = pd.read_csv(f"/gallery_louvre/dayoon.ko/research/sds/src/datasets/{dataset_name}/qrels/{mode}.tsv", sep="\t")
qrels = qrels[qrels["score"] >= 1].sort_values("corpus-id")
#print(len(qrels["corpus-id"].unique()) - len(qrels["corpus-id"]))
qrels

,query-id,corpus-id,score
983,699,1257_Samalas_eruption,1
990,702,1257_Samalas_eruption,1
2329,1579,1257_Samalas_eruption,1
4229,2826,1257_Samalas_eruption,1
572,424,1906_San_Francisco_earthquake,1
...,...,...,...
2960,1971,Zero-energy_building,1
520,382,Zero-energy_building,1
2854,1893,Zika_fever,1
1250,871,Zika_fever,1


In [577]:
# Time consuming
gt = {}
for k in qrels["corpus-id"].unique():
    qrels_ = qrels[qrels["corpus-id"] == k]
    v = [str(i) for i in qrels_["query-id"].tolist()]
    gt[str(k)] = v

In [578]:
results[0]["retrieval"][0]

{'source': '/gallery_louvre/dayoon.ko/research/sds/src/datasets/climate-fever/corpus.jsonl',
 'seq_num': 286229,
 'document': 'The Agreement on the Conservation of Polar Bears is a multilateral treaty signed in Oslo , November 15 , 1973 by the five nations with the largest polar bear populations : Canada , Denmark ( Greenland ) , Norway ( Svalbard ) , the United States , and the Soviet Union . This treaty was brought about due to increased hunting of polar bears during the 1960s and 1970s which led to polar bears being under severe survival pressure from hunters .   The agreement prohibits random , unregulated sport hunting of polar bears and outlaws hunting of polar bears from aircraft and icebreakers which have been the most detrimental to the polar bear population . Additionally , the agreement holds member states accountable for taking appropriate actions to protect the ecosystems of which the polar bears dwell , paying special attention to places where polar bears create dens , do

### Model for retrieval

In [646]:
model_name = ["multilingual-e5-large", "bge-large-en-v1.5", "contriever", "gte-base"][0]
mode = "test"
with open(f"{model_name}/{dataset_name}.jsonl") as f:
    results = [json.loads(i) for i in f.readlines()]
# Get retrieval results as corpus id
retrieval_results = {}
for res in results:
    qid = res["_id"]
    for retrieval in res["retrieval"]:
        doc_id = retrieval["_id"]
        if doc_id in retrieval_results:
            retrieval_results[doc_id].append(qid)
        else:
            retrieval_results[doc_id] = [qid]

In [647]:
model_name, dataset_name, mode

('multilingual-e5-large', 'climate-fever', 'test')

In [648]:
print("Total", len(qrels["corpus-id"].unique()), "documents!")

Total 1344 documents!


### Calculate metrics

In [649]:
corpus_ids = []
num_gts = []
recall = [] # retrieval 돼야하는데 된 경우 / retrieval 돼야하는 경우 (gt)
precision = [] # retrieval 돼야하는데 된 경우 / retrieval 이 된 경우 (ret)
for k, gt_v in gt.items():
    gt_v = [str(i) for i in gt_v]
    v = retrieval_results[k] if k in retrieval_results else []
    # gt_v: document 가 retrieval 되어야하는 모든 쿼리 / v: document 가 실제 retrieval 된 쿼리
    tp_t = len(set(gt_v).intersection(set(v))) / len(gt_v) # True Positive / # True
    tp_p = len(set(gt_v).intersection(set(v))) / len(v) if len(v) > 0 else 0 # True Positive / # Positive
    corpus_ids.append(k)
    num_gts.append(len(gt_v))
    recall.append(tp_t)
    precision.append(tp_p)

In [650]:
results[0]

{'_id': '0',
 'text': 'Global warming is driving polar bears toward extinction',
 'metadata': {'claim_label': 'SUPPORTS',
  'evidence_label': None,
  'evidences': [{'evidence_id': 'Extinction risk from global warming:170',
    'evidence_label': 'NOT_ENOUGH_INFO',
    'article': 'Extinction risk from global warming',
    'evidence': '"Recent Research Shows Human Activity Driving Earth Towards Global Extinction Event".',
    'entropy': 0.6931471805599453,
    'votes': ['SUPPORTS', 'NOT_ENOUGH_INFO', None, None, None]},
   {'evidence_id': 'Global warming:14',
    'evidence_label': 'SUPPORTS',
    'article': 'Global warming',
    'evidence': 'Environmental impacts include the extinction or relocation of many species as their ecosystems change, most immediately the environments of coral reefs, mountains, and the Arctic.',
    'entropy': 0.0,
    'votes': ['SUPPORTS', 'SUPPORTS', None, None, None]},
   {'evidence_id': 'Global warming:178',
    'evidence_label': 'NOT_ENOUGH_INFO',
    'articl

In [651]:
data = {
        "corpus-id": corpus_ids, 
        "n-query": num_gts,
        "recall": recall,
        "precision": precision
       }
df = pd.DataFrame(data)
df = df[df["n-query"] >= 3]
df.sort_values("recall")

,corpus-id,n-query,recall,precision
933,Planetary_boundaries,4,0.0,0.000000
1161,Sustainable_energy,3,0.0,0.000000
999,Republican_Party_(United_States),3,0.0,0.000000
245,Climate_of_India,3,0.0,0.000000
561,Holocene_extinction,9,0.0,0.000000
...,...,...,...,...
290,Coral_reef,10,1.0,0.322581
413,Environmental_policy_of_the_Donald_Trump_admin...,3,1.0,0.176471
407,Environmental_impact_of_aviation,4,1.0,0.133333
520,Great_Barrier_Reef,13,1.0,0.541667


In [652]:
df["f1"] = [0 if i < 0.0001 and j < 0.0001 else 2 * i * j / (i + j) for i, j in zip(df.recall, df.precision)]
df = df.sort_values("f1", ascending=False)
df

,corpus-id,n-query,recall,precision,f1
637,Jet_stream,7,1.00,0.875000,0.933333
1270,Venus,5,0.80,1.000000,0.888889
344,Donald_Trump,5,0.80,1.000000,0.888889
943,Polar_bear,11,1.00,0.785714,0.880000
0,1257_Samalas_eruption,4,0.75,0.750000,0.750000
...,...,...,...,...,...
1096,Smart_grid,3,0.00,0.000000,0.000000
933,Planetary_boundaries,4,0.00,0.000000,0.000000
1104,Solar_System,3,0.00,0.000000,0.000000
835,New_York_City,5,0.00,0.000000,0.000000


In [653]:
df[df["recall"] > 0.2].sort_values("recall", ascending=False)

,corpus-id,n-query,recall,precision,f1
637,Jet_stream,7,1.000000,0.875000,0.933333
290,Coral_reef,10,1.000000,0.322581,0.487805
381,Effects_of_global_warming_on_humans,5,1.000000,0.016835,0.033113
1087,Shutdown_of_thermohaline_circulation,5,1.000000,0.020161,0.039526
604,Ice_shelf,3,1.000000,0.029412,0.057143
...,...,...,...,...,...
1003,Richard_Lindzen,9,0.222222,1.000000,0.363636
78,An_Inconvenient_Truth,9,0.222222,0.666667,0.333333
1211,Tipping_points_in_the_climate_system,9,0.222222,0.117647,0.153846
229,Climate_change_in_Australia,9,0.222222,0.046512,0.076923


In [654]:
#cid = "MED-2662"
#qrels[qrels["corpus-id"] == cid]

In [655]:
#print("Corpus")
#print(corpus_dict[cid])
#print("\n")
#print("Queries")
#for i in qrels[qrels["corpus-id"] == cid]["query-id"].tolist():
#    print(queries_dict[str(i)])


In [656]:
#corpus_dict["3h1o0oz3"]

# Generate dataset

In [657]:
for i in range(1, len(df["n-query"].unique())+1):
    print(f"n_query=={i} : total {len(df[df['n-query'] == i])} datapoints")

n_query==1 : total 0 datapoints
n_query==2 : total 0 datapoints
n_query==3 : total 70 datapoints
n_query==4 : total 41 datapoints
n_query==5 : total 31 datapoints
n_query==6 : total 19 datapoints
n_query==7 : total 19 datapoints
n_query==8 : total 7 datapoints
n_query==9 : total 15 datapoints
n_query==10 : total 10 datapoints
n_query==11 : total 6 datapoints
n_query==12 : total 5 datapoints
n_query==13 : total 2 datapoints
n_query==14 : total 6 datapoints
n_query==15 : total 3 datapoints
n_query==16 : total 4 datapoints
n_query==17 : total 1 datapoints
n_query==18 : total 1 datapoints
n_query==19 : total 3 datapoints
n_query==20 : total 1 datapoints
n_query==21 : total 1 datapoints
n_query==22 : total 2 datapoints
n_query==23 : total 1 datapoints
n_query==24 : total 3 datapoints
n_query==25 : total 1 datapoints
n_query==26 : total 1 datapoints
n_query==27 : total 0 datapoints
n_query==28 : total 1 datapoints
n_query==29 : total 0 datapoints
n_query==30 : total 2 datapoints
n_query==31 

In [658]:
for i in range(11):
    i = round(i / 10, 2)
    j = round(i + 0.1, 2)
    print(f"{i}<=recall<{j} : total {len(df[(df['recall'] >= i) & (df['recall'] < j)])} datapoints")

0.0<=recall<0.1 : total 62 datapoints
0.1<=recall<0.2 : total 15 datapoints
0.2<=recall<0.3 : total 31 datapoints
0.3<=recall<0.4 : total 36 datapoints
0.4<=recall<0.5 : total 15 datapoints
0.5<=recall<0.6 : total 20 datapoints
0.6<=recall<0.7 : total 29 datapoints
0.7<=recall<0.8 : total 21 datapoints
0.8<=recall<0.9 : total 19 datapoints
0.9<=recall<1.0 : total 6 datapoints
1.0<=recall<1.1 : total 21 datapoints


In [659]:
df.to_csv(f"{model_name}/{dataset_name}-n-query-mt-2.csv")

In [660]:
f"{model_name}/{dataset_name}-n-query-mt-2.csv"

'multilingual-e5-large/climate-fever-n-query-mt-2.csv'

In [644]:
# hotpotqa: 5233329
# trec-covid: 173???
# np: 2681468

In [188]:
465 rows × 3 columns / 1983

SyntaxError: invalid character '×' (U+00D7) (2594108381.py, line 1)